In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# The List of 20 type of cryptocurrency
coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']


In [3]:
#defining the dataframe
main_df = pd.DataFrame()

In [9]:
%%time
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])
    
    # Defining the Start Date and End Date
    datetime_end = datetime(2022, 7, 9, 0, 0)
    datetime_check = datetime(2022, 7, 8, 0, 0)
    
    while len(df) > 0:
        if datetime_end == datetime_check:
            break
        
        datetime_start = datetime_end - relativedelta(hours = 12)
        
        #Api for the scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+datetime_start.strftime("%Y-%m-%dT%H:%M") + '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'
        
        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        
        # To handle the Missing Data
        insert_ids_list = [np.nan]
        
        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data. 
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will 
        be substituted for the missing one.
        '''
        
        while len(insert_ids_list) > 0:
            timestamp_checking = np.array(df['Timestamp'][1:]) - np.array(df['Timestamp'][:-1])
            insert_ids_list = np.where(timestamp_checking!= 60000)[0]
            if len(insert_ids_list) > 0:
                print(str(len(insert_ids_list)) + ' mismatched.')
                insert_ids = insert_ids_list[0]
                temp_df = df.iloc[insert_ids.repeat(int(timestamp_checking[insert_ids]/60000)-1)].reset_index(drop=True)
                temp_df['Timestamp'] = [temp_df['Timestamp'][0] + i*60000 for i in range(1, len(temp_df)+1)]
                df = df.loc[:insert_ids].append(temp_df).append(df.loc[insert_ids+1:]).reset_index(drop=True)
                insert_ids_list = insert_ids_list[1:]
                
        
        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
        coin_df = df.append(coin_df)
        datetime_end = datetime_start
        
    coin_df['Symbol'] = coin
    main_df = main_df.append(coin_df)

33 mismatched.
32 mismatched.
31 mismatched.
30 mismatched.
29 mismatched.
28 mismatched.
27 mismatched.
26 mismatched.
25 mismatched.
24 mismatched.
23 mismatched.
22 mismatched.
21 mismatched.
20 mismatched.
19 mismatched.
18 mismatched.
17 mismatched.
16 mismatched.
15 mismatched.
14 mismatched.
13 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
2 mismatched.
1 mismatched.
33 mismatched.
32 mismatched.
31 mismatched.
30 mismatched.
29 mismatched.
28 mismatched.
27 mismatched.
26 mismatched.
25 mismatched.
24 mismatched.
23 mismatched.
22 mismatched.
21 mismatched.
20 mismatched.
19 mismatched.
18 mismatched.
17 mismatched.
16 mismatched.
15 mismatched.
14 mismatched.
13 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.


2 mismatched.
1 mismatched.
33 mismatched.
32 mismatched.
31 mismatched.
30 mismatched.
29 mismatched.
28 mismatched.
27 mismatched.
26 mismatched.
25 mismatched.
24 mismatched.
23 mismatched.
22 mismatched.
21 mismatched.
20 mismatched.
19 mismatched.
18 mismatched.
17 mismatched.
16 mismatched.
15 mismatched.
14 mismatched.
13 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
2 mismatched.
1 mismatched.
33 mismatched.
32 mismatched.
31 mismatched.
30 mismatched.
29 mismatched.
28 mismatched.
27 mismatched.
26 mismatched.
25 mismatched.
24 mismatched.
23 mismatched.
22 mismatched.
21 mismatched.
20 mismatched.
19 mismatched.
18 mismatched.
17 mismatched.
16 mismatched.
15 mismatched.
14 mismatched.
13 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.


In [10]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df


,Datetime,Symbol,Open,High,Low,Close
0,2022-07-08 00:00:00,BTC,21612.419739,21649.285912,21612.419739,21644.446463
1,2022-07-08 00:01:00,BTC,21644.689092,21652.627832,21630.575677,21630.585442
2,2022-07-08 00:02:00,BTC,21629.220120,21629.220120,21610.168638,21624.434695
3,2022-07-08 00:03:00,BTC,21624.881043,21640.246861,21614.894978,21619.035132
4,2022-07-08 00:04:00,BTC,21616.852120,21616.852120,21585.112490,21597.373574
...,...,...,...,...,...,...
27355,2022-07-08 23:55:00,EOS,1.021343,1.021428,1.020770,1.020826
27356,2022-07-08 23:56:00,EOS,1.020829,1.021502,1.015601,1.016211
27357,2022-07-08 23:57:00,EOS,1.016189,1.016189,1.013683,1.013724
27358,2022-07-08 23:58:00,EOS,1.013691,1.014559,1.011891,1.012636


In [11]:
main_df.shape

(27360, 6)

In [13]:
main_df.to_csv('main_df.csv', index=False)